# Workshop 5: Clustering and Dimensionality Reduction
## Student Practice Notebook

This notebook accompanies the tutor's live demonstration. Run the cells as the tutor presents, and try the **"Your turn"** variations to reinforce your understanding.

**Topics covered:**
1. K-Means Clustering
2. Evaluating Clusters (Elbow Method & Silhouette Score)
3. PCA (Principal Component Analysis)

---

# Part 1: K-Means Clustering

## 1.1 Create toy data

We'll create a simple dataset with 3 natural clusters to demonstrate how K-Means works.

In [ ]:
# Toy data: a small, simple dataset used to illustrate a concept
# Here: we create random points that naturally form 3 clusters

import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# Define the original cluster centres. We'll compare these to what K-Means finds
original_centres = np.array([
    [2, 2],  # Cluster A
    [6, 6],  # Cluster B
    [2, 6]   # Cluster C
])

# Create 3 clusters of points, each with a different centre
cluster1 = np.random.randn(30, 2) * 0.6 + original_centres[0]
cluster2 = np.random.randn(30, 2) * 0.6 + original_centres[1]
cluster3 = np.random.randn(30, 2) * 0.6 + original_centres[2]

# Combine all points into one dataset
X = np.vstack([cluster1, cluster2, cluster3])

print(f"Dataset shape: {X.shape}")
print(f"This means we have {X.shape[0]} data points, each with {X.shape[1]} features")
print(f"\nOriginal centres used to generate the data:")
for i, centre in enumerate(original_centres):
    print(f"  Cluster {chr(65+i)}: ({centre[0]:.2f}, {centre[1]:.2f})")

## 1.2 Visualise the unlabelled data

This is what K-Means "sees": points with no labels. Can you spot the clusters?

In [ ]:
# Visualise the UNLABELLED data. this is what clustering algorithms see

plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c='steelblue', s=50, alpha=0.7)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Unlabelled Data. Can You See the Clusters?')
plt.grid(True, alpha=0.3)
plt.show()

print("Key insight: Clustering is UNSUPERVISED. We don't tell the algorithm where the groups are.")

## 1.3 Apply K-Means (k=3)

Now we'll use K-Means to find the clusters automatically.

In [ ]:
# K-Means in action: let's cluster our toy data
from sklearn.cluster import KMeans

# Create a K-Means model with 3 clusters
# n_clusters is a HYPERPARAMETER: we set it, the algorithm doesn't learn it
kmeans = KMeans(n_clusters=3, random_state=42)

# fit_predict: learns cluster centres and assigns each point to a cluster
labels = kmeans.fit_predict(X)

# The cluster centres (centroids) that K-Means found
centres = kmeans.cluster_centers_

print(f"Cluster labels assigned: {np.unique(labels)}")
print(f"\nK-Means found these centres:")
for i, centre in enumerate(centres):
    print(f"  Cluster {i}: ({centre[0]:.2f}, {centre[1]:.2f})")

# Compare to the original centres we used to generate the data
print(f"\nOriginal centres (for comparison):")
for i, centre in enumerate(original_centres):
    print(f"  Cluster {chr(65+i)}: ({centre[0]:.2f}, {centre[1]:.2f})")

print("\n→ The algorithm recovered centres very close to the originals!")
print("  Slight differences are due to random variation in the data points.")

In [ ]:
# Visualise the clustering result
plt.figure(figsize=(10, 6))

# Plot points coloured by their assigned cluster
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=50, alpha=0.7)

# Plot the cluster centres as red X markers
plt.scatter(centres[:, 0], centres[:, 1], c='red', marker='X', s=200, 
            edgecolors='black', linewidths=2, label='Centroids')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('K-Means Clustering Result (k=3)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Each point is assigned to its NEAREST centroid.")

### Your turn: Try different values of k

What happens if we ask K-Means to find a different number of clusters?

In [ ]:
# YOUR TURN: Change n_clusters to 2, then to 4, and observe what happens
# How does the clustering change? Does k=2 or k=4 make sense for this data?

kmeans_try = KMeans(n_clusters=2, random_state=42)  # <-- Try changing this to 2 or 4
labels_try = kmeans_try.fit_predict(X)
centres_try = kmeans_try.cluster_centers_

# Visualise
plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 1], c=labels_try, cmap='viridis', s=50, alpha=0.7)
plt.scatter(centres_try[:, 0], centres_try[:, 1], c='red', marker='X', s=200, 
            edgecolors='black', linewidths=2, label='Centroids')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title(f'K-Means Clustering Result (k={len(centres_try)})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

# Part 2: Evaluating Clusters

## 2.1 The Elbow Method (WCSS)

How do we choose the right number of clusters? The **elbow method** plots WCSS (Within-Cluster Sum of Squares) for different values of k.

In [ ]:
# The Elbow Method: finding the optimal number of clusters
# WCSS = how spread out points are within their clusters (lower = tighter clusters)

k_values = range(1, 11)  # Try k from 1 to 10
wcss_values = []  # Store WCSS for each k

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    # inertia_ is sklearn's name for WCSS
    wcss_values.append(kmeans.inertia_)

print("WCSS for each k:")
for k, wcss in zip(k_values, wcss_values):
    print(f"  k={k}: WCSS = {wcss:.1f}")

In [ ]:
# Plot the Elbow curve
plt.figure(figsize=(10, 6))
plt.plot(k_values, wcss_values, 'bo-', linewidth=2, markersize=8)

# Highlight the elbow point (k=3 in our case)
plt.axvline(x=3, color='red', linestyle='--', alpha=0.7, label='Elbow point (k=3)')

plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('WCSS (Within-Cluster Sum of Squares)', fontsize=12)
plt.title('Elbow Method: Finding Optimal k', fontsize=14)
plt.xticks(k_values)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("The 'elbow' is where adding more clusters stops giving big improvements.")
print("Here, the elbow is at k=3.")

## 2.2 Silhouette Score

Another way to evaluate clustering quality. Measures how similar points are to their own cluster vs other clusters.

- **+1** = perfect clustering
- **0** = overlapping clusters  
- **-1** = points in wrong cluster

In [ ]:
# Silhouette Score — another way to evaluate clustering quality
# Measures how similar points are to their own cluster vs other clusters
# Range: -1 (bad) to +1 (good), 0 means overlapping clusters

# How it works (for each point):
#   a = avg distance to points in SAME cluster (cohesion)
#   b = avg distance to points in NEAREST OTHER cluster (separation)
#   silhouette = (b - a) / max(a, b)
# If b >> a → score near +1 (good), if a >> b → score near -1 (bad)

from sklearn.metrics import silhouette_score

# Calculate silhouette score for different values of k
# Note: silhouette score requires at least 2 clusters
k_values_sil = range(2, 11)
silhouette_values = []

for k in k_values_sil:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X)
    score = silhouette_score(X, labels)
    silhouette_values.append(score)

print("Silhouette scores for each k:")
for k, score in zip(k_values_sil, silhouette_values):
    print(f"  k={k}: Silhouette = {score:.3f}")

In [ ]:
# Plot the Silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(list(k_values_sil), silhouette_values, 'go-', linewidth=2, markersize=8)

# Highlight the best k (highest silhouette score)
best_k = list(k_values_sil)[np.argmax(silhouette_values)]
best_score = max(silhouette_values)
plt.axvline(x=best_k, color='red', linestyle='--', alpha=0.7, 
            label=f'Best k={best_k} (score={best_score:.3f})')

plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Silhouette Method: Finding Optimal k', fontsize=14)
plt.xticks(list(k_values_sil))
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nHighest silhouette score is {best_score:.3f} at k={best_k}")
print("Higher is better: both methods agree that k=3 is optimal.")

### Your turn: Interpret the results

Look at the elbow plot and silhouette scores above:
- Do both methods agree on the optimal k?
- What would happen if they disagreed?

---

# Part 3: PCA (Principal Component Analysis)

## 3.1 Create correlated data

PCA is useful when features are correlated (contain redundant information). Let's create some data to demonstrate.

In [ ]:
# PCA Demo: reducing dimensions while keeping the important information
# First, let's create some correlated data with 4 features

import pandas as pd

np.random.seed(42)

# Create a base variable
base = np.random.randn(100)

# Create 4 features: some are correlated with each other
feature1 = base + np.random.randn(100) * 0.2          # Strongly related to base
feature2 = base * 0.8 + np.random.randn(100) * 0.3    # Also related to base
feature3 = np.random.randn(100)                        # Independent
feature4 = feature3 * 0.5 + np.random.randn(100) * 0.4  # Related to feature3

# Combine into a DataFrame
df_pca = pd.DataFrame({
    'Feature1': feature1,
    'Feature2': feature2,
    'Feature3': feature3,
    'Feature4': feature4
})

print(f"Dataset shape: {df_pca.shape}")
print(f"\nFirst 5 rows:")
df_pca.head()

## 3.2 Check correlations

A correlation heatmap shows which features are related to each other.

In [ ]:
# Check correlations between features
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(df_pca.corr(), annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

print("Notice: Features 1 and 2 are very highly correlated (~0.97),")
print("and Features 3 and 4 are also strongly correlated (~0.79).")
print("PCA can help reduce this redundancy.")

## 3.3 Apply PCA

We'll reduce from 4 features to 2 principal components.

**Important:** Always scale your data before PCA!

In [ ]:
# Apply PCA to reduce from 4 dimensions to 2
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Step 1: Standardise the data (important for PCA!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_pca)

# Step 2: Apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f"Original shape: {df_pca.shape}")
print(f"After PCA:      {X_pca.shape}")
print(f"\nWe reduced from 4 features to 2 principal components!")

## 3.4 How much information did we keep?

In [ ]:
# explained_variance_ratio_ tells us how much variance each component captures

print("Variance explained by each principal component:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.1%}")

total_var = sum(pca.explained_variance_ratio_)
print(f"\nTotal variance explained by 2 components: {total_var:.1%}")
print(f"\nWe kept {total_var:.1%} of the information while halving the number of features!")
print("\nThis makes sense: we had two pairs of correlated features (1-2 and 3-4),")
print("so two components can capture the essential structure.")

## 3.5 Visualise in PCA space

In [ ]:
# Visualise the data in the new 2D PCA space
plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c='steelblue', s=50, alpha=0.7)
plt.xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
plt.ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
plt.title('Data Projected onto First 2 Principal Components', fontsize=14)
plt.axhline(y=0, color='grey', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='grey', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.show()

print("PC1 captures the main direction of variation in the data.")
print("PC2 captures the second most important direction (perpendicular to PC1).")

### Your turn: Try different numbers of components

What if we only keep 1 component? Or keep 3?

In [ ]:
# YOUR TURN: Change n_components to 1 or 3 and see how variance explained changes

pca_try = PCA(n_components=1)  # <-- Try changing this to 1 or 3
X_pca_try = pca_try.fit_transform(X_scaled)

print(f"Components: {pca_try.n_components_}")
print(f"Shape after PCA: {X_pca_try.shape}")
print(f"\nVariance explained by each component:")
for i, var in enumerate(pca_try.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.1%}")
print(f"\nTotal variance explained: {sum(pca_try.explained_variance_ratio_):.1%}")

---

# Quick Reference

## K-Means Syntax

```python
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(X)

kmeans.cluster_centers_  # Centroid coordinates
kmeans.inertia_          # WCSS
```

## Evaluation Syntax

```python
from sklearn.metrics import silhouette_score

score = silhouette_score(X, labels)  # -1 to +1, higher is better
```

## PCA Syntax

```python
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Always scale first!
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

pca.explained_variance_ratio_  # Variance per component
```